In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$spark_version/$spark_version-bin-hadoop3.tgz
!tar xf $spark_version-bin-hadoop3.tgz
!pip install -q findspark


# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [78.0 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,252 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,351 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubunt

In [12]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark.sql.functions import sum, avg, col

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [5]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
home_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales.show()
home_sales.createOrReplaceTempView('home_sales')

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [13]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
home_sales.groupBy("bedrooms") \
    .agg(avg("price").alias("avg_price")) \
    .where(col("bedrooms") == 4) \
    .show(truncate=False)

+--------+-----------------+
|bedrooms|avg_price        |
+--------+-----------------+
|4       |299661.0056283447|
+--------+-----------------+



In [17]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
home_sales.groupBy("bedrooms", "bathrooms") \
    .agg(avg("price").alias("avg_price")) \
    .where(col("bedrooms") == 3) \
    .where(col("bathrooms") == 3) \
    .show(truncate=False)

+--------+---------+------------------+
|bedrooms|bathrooms|avg_price         |
+--------+---------+------------------+
|3       |3        |292003.60507783544|
+--------+---------+------------------+



In [18]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
home_sales.groupBy("bedrooms", "bathrooms", "floors", "sqft_living") \
    .agg(avg("price").alias("avg_price")) \
    .where(col("bedrooms") == 3) \
    .where(col("bathrooms") == 3) \
    .where(col("floors") == 2) \
    .where(col("sqft_living") >= 2000) \
    .show(truncate=False)

+--------+---------+------+-----------+------------------+
|bedrooms|bathrooms|floors|sqft_living|avg_price         |
+--------+---------+------+-----------+------------------+
|3       |3        |2     |2005       |246760.0          |
|3       |3        |2     |2143       |243386.66666666666|
|3       |3        |2     |2080       |167351.5          |
|3       |3        |2     |2292       |332279.0          |
|3       |3        |2     |2002       |215001.66666666666|
|3       |3        |2     |2489       |375258.0          |
|3       |3        |2     |2091       |274173.0          |
|3       |3        |2     |2314       |294597.3333333333 |
|3       |3        |2     |2040       |314311.3333333333 |
|3       |3        |2     |2441       |226803.66666666666|
|3       |3        |2     |2364       |329629.6666666667 |
|3       |3        |2     |2349       |188709.33333333334|
|3       |3        |2     |2469       |281876.5          |
|3       |3        |2     |2276       |287241.5         

In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00010347366333007812 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
spark.sql("select date_built from home_sales").show()

+----------+
|date_built|
+----------+
|      2016|
|      2013|
|      2014|
|      2016|
|      2017|
|      2017|
|      2017|
|      2010|
|      2016|
|      2015|
|      2011|
|      2014|
|      2015|
|      2016|
|      2011|
|      2011|
|      2016|
|      2015|
|      2015|
|      2016|
+----------+
only showing top 20 rows



In [ ]:
# 11. Read the parquet formatted data.
# Write out the data in parquet format
df.write.parquet('home_sales', mode='overwrite')


In [ ]:
# 12. Create a temporary table for the parquet data.
p_df=spark.read.parquet('home_sales')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00014638900756835938 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached

# Verify that the table is no longer cached
if spark.catalog.isCached("home_sales"):
    print("a table is till cached")
else:
    print("all clear")

all clear
